## 37 Renderizando uma textura por completo

As imagens usadas neste notebook foram obtidas nos sites:

https://www.pngwing.com/

https://www.pngfuel.com

In [9]:
import glm
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v3 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v4 import ModelRenderer
from cg.models.SquareMesh_v3 import SquareMesh
from cg.utils.Texture import Texture

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraPos   = glm.vec3(0.0, 0.0, 1.2)
        self.cameraFront = glm.vec3(0.0, 0.0, 0.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
    
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        # define uma malha quadrada
        vertex_position = np.array([
            -0.50, -0.50, 0.0, 1.0, # Triângulo 1
             0.50, -0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0,
             0.50, -0.50, 0.0, 1.0, # Triângulo 2
             0.50,  0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0],
            dtype=np.float32)
        
        # define as coordendas de texturas para renderizá-la por completo
        vertex_tex = np.array([
             0.00,  1.00, # Triângulo 1
             1.00,  1.00,
             0.00,  0.00,
             1.00,  1.00, # Triângulo 2
             1.00,  0.00,
             0.00,  0.00],
            dtype=np.float32)
        
        # carrega e cria as texturas
        self.texture_1 = Texture('./cg/images/textures/tex01_pngwing.com.png')
        self.texture_2 = Texture('./cg/images/textures/tex02_pngwing.com.png')
        self.texture_3 = Texture('./cg/images/textures/tex03_pngwing.com.png')
        self.texture_4 = Texture('./cg/images/textures/tex04_pngfuel.com.png')
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(vertex_position, vertex_tex=vertex_tex)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        tex_coord_loc = self.shaderProgram.getVertexTextureCoordLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)
        self.squareRenderer.setVertexTextureCoordLoc(tex_coord_loc)

        # inicializa a variável que contém o ângulo de rotação
        self.angle = 0
        
        # configura a cor de background
        gl.glClearColor(0.9, 0.9, 0.9, 1)
        
        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        #configura a função de blending
        gl.glBlendFunc(gl.GL_SRC_ALPHA, gl.GL_ONE_MINUS_SRC_ALPHA)
        
    def paintGL(self):
        
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # incrementa a variável que contém o ângulo de rotação
        self.angle += 0.1
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza os quadrados texturizados
        self.renderTexture2()
        self.renderTexture3()
        self.renderTexture4()
        self.renderTexture1()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
        
    def renderTexture1(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        transl = glm.translate(glm.mat4(), glm.vec3(-0.7, -1.05, -1.5))
        model_matrix = glm.rotate(transl, glm.radians(-self.angle), glm.vec3(0.0, 0.0, 1.0))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_1.getTextureID())
        
        # renderiza
        self.squareRenderer.render()
    
        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        transl = glm.translate(glm.mat4(), glm.vec3(0.7, -1.05, -1.5))
        model_matrix = glm.rotate(transl, glm.radians(-self.angle), glm.vec3(0.0, 0.0, 1.0))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # renderiza com transparência
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)
        
    def renderTexture2(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.7, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_2.getTextureID())
        
        # renderiza
        self.squareRenderer.render()

        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(1.8, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # renderiza com transparência
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)
        
    def renderTexture3(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-1.8, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_3.getTextureID())
        
        # renderiza
        self.squareRenderer.render()
        
        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-0.7, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # renderiza com transparência
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)
        
    def renderTexture4(self):
        
        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader        
        transl = glm.translate(glm.mat4(), glm.vec3(0.0, 0.0, 0.99))
        model_matrix = glm.scale(transl, glm.vec3(2.0, 2.0, 2.0))
        self.shaderProgram.setUniformMVPMatrix(model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_4.getTextureID())
        
        # renderiza com transparência
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)

    def resizeGL(self, width, height):
        
        gl.glViewport(0, 0, width, height)
        
        # configura a projeção
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 50.0)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(1200, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

trying to open ./cg/images/textures/tex01_pngwing.com.png
opened file: size= (1000, 1000) format= PNG mode= RGBA
trying to open ./cg/images/textures/tex02_pngwing.com.png
opened file: size= (650, 650) format= PNG mode= RGBA
trying to open ./cg/images/textures/tex03_pngwing.com.png
opened file: size= (650, 650) format= PNG mode= RGBA
trying to open ./cg/images/textures/tex04_pngfuel.com.png
opened file: size= (658, 774) format= PNG mode= RGBA


SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
! jupyter nbconvert --to python 37_Renderizando_uma_textura_por_completo.ipynb
%run -i 37_Renderizando_uma_textura_por_completo.py

[NbConvertApp] Converting notebook 37_Quadrado_com_texturas.ipynb to python
[NbConvertApp] Writing 8073 bytes to 37_Quadrado_com_texturas.py


trying to open ./pngwing.com_1.png
opened file: size= (650, 650) format= None mode= RGBA
trying to open ./pngwing.com_2.png
opened file: size= (1000, 1000) format= None mode= RGBA
trying to open ./pngwing.com_4.png
opened file: size= (650, 650) format= None mode= RGBA
